In [ ]:
!pip install geopandas

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 14.8MB 309kB/s 
     |████████████████████████████████| 10.9MB 42.5MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:

alldata = []
directory = 'dataset/' #dataset directory
for filename in os.listdir(directory):
    if filename.endswith(".csv") and filename.startswith("atp"):
        print(os.path.join(directory, filename))
        year= filename.split("_")[2].split(".")[0]  #Getting year from filename to add in dataframe in Year column
        file1 = open(os.path.join(directory, filename), 'r')
        Lines = file1.readlines() 
        file_list = []
        for line in Lines: 
            list_lin = line.rstrip("\n").rstrip(',').split(",") #Stripping out additional comma at the end of line
            file_list.append(list_lin) #appending cleaned lies to a temp list 
        tempdf=pd.DataFrame(file_list[1:],columns=file_list[0]) #converting temp list (excluding first line as headings) into dataframe
        tempdf['year']= int(year) #Adding year into dataframe columns
        alldata.append(tempdf) #adding dataframe to a list
    else:
        continue
tennis_df = pd.concat(alldata) #Concatenating all data frame
tennis_df.reset_index(drop=True,inplace=True) #Resetting index




dataset/atp_matches_2010.csv
dataset/atp_matches_2002.csv
dataset/atp_matches_2016.csv
dataset/atp_matches_2014.csv
dataset/atp_matches_2000.csv
dataset/atp_matches_2006.csv
dataset/atp_matches_2008.csv
dataset/atp_matches_2001.csv
dataset/atp_matches_2017.csv
dataset/atp_matches_2003.csv
dataset/atp_matches_2007.csv
dataset/atp_matches_2009.csv
dataset/atp_matches_2015.csv
dataset/atp_matches_2012.csv
dataset/atp_matches_2013.csv
dataset/atp_matches_2005.csv
dataset/atp_matches_2011.csv
dataset/atp_matches_2004.csv


In [ ]:
tennis_df.tail()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,winner_rank,winner_rank_points,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,loser_rank,loser_rank_points,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,year
53566,2004-D082,Davis Cup WG PO: ROU vs CAN,Clay,4,D,20040924,4,104181,,,Victor Ionita,R,185,ROU,21.363449692,216,176,103225,,,Simon Larose,R,190,CAN,26.2313483915,301,105,4-2 RET,5,RR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2004
53567,2004-D083,Davis Cup WG PO: RUS vs THA,Clay,4,D,20040924,1,104214,,,Igor Andreev,R,185,RUS,21.1882272416,40,826,103387,,,Paradorn Srichaphan,R,185,THA,25.2703627652,20,1280,7-5 6-2 6-4,5,RR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2004
53568,2004-D083,Davis Cup WG PO: RUS vs THA,Clay,4,D,20040924,2,103498,,,Marat Safin,R,193,RUS,24.6488706366,9,1760,103821,,,Danai Udomchoke,R,173,THA,23.1101984942,163,245,6-4 6-1 6-2,5,RR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2004
53569,2004-D083,Davis Cup WG PO: RUS vs THA,Clay,4,D,20040924,3,103786,,,Nikolay Davydenko,R,178,RUS,23.3018480493,45,779,103921,,,Sonchat Ratiwatana,R,175,THA,22.6584531143,736,17,6-2 6-0,3,RR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2004
53570,2004-D083,Davis Cup WG PO: RUS vs THA,Clay,4,D,20040924,4,104022,,,Mikhail Youzhny,R,183,RUS,22.2395619439,33,1005,103920,,,Sanchai Ratiwatana,R,175,THA,22.6584531143,904,8,6-4 6-2,3,RR,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2004


In [ ]:
# checking if all csvs have same columns
'''prev_list = ''
for x in alldata:
    if prev_list == '':
        prev_list=str(x.columns)
    else:
        if str(x.columns) == prev_list:
            print("matches")
            prev_list=str(x.columns)
        else:
            print("not macthes",str(x.columns))
            break'''

'prev_list = \'\'\nfor x in alldata:\n    if prev_list == \'\':\n        prev_list=str(x.columns)\n    else:\n        if str(x.columns) == prev_list:\n            print("matches")\n            prev_list=str(x.columns)\n        else:\n            print("not macthes",str(x.columns))\n            break'

In [ ]:
#Missing values in winne_ioc columns
sum(tennis_df['winner_ioc'].isna())

63

In [ ]:
len(tennis_df['winner_ioc'])

53571

In [ ]:
tennis_df[['year','winner_ioc']].head()

,year,winner_ioc
0,2010,GER
1,2010,USA
2,2010,ARG
3,2010,SUI
4,2010,NZL


In [ ]:
print("Unique countries",len(set(list(tennis_df['winner_ioc']))))

Unique countries 105


In [ ]:
#viewing sample countries
for x in alldata:
    print(np.unique(list(x['year'])))
    print(list(x['winner_ioc'].head()))
    print("-------------------------------")

[2010]
['GER', 'USA', 'ARG', 'SUI', 'NZL']
-------------------------------
[2002]
['USA', 'KOR', 'FRA', 'AUT', 'USA']
-------------------------------
[2016]
['CAN', 'SUI', 'CAN', 'SUI', 'AUT']
-------------------------------
[2014]
['USA', 'COL', 'ESP', 'USA', 'TPE']
-------------------------------
[2000]
['FRA', 'CHI', 'THA', 'NED', 'AUS']
-------------------------------
[2006]
['SUI', 'COL', 'BLR', 'CHI', 'CZE']
-------------------------------
[2008]
['FRA', 'RUS', 'FRA', 'ESP', 'FRA']
-------------------------------
[2001]
['BRA', 'GBR', 'AUT', 'GER', 'FRA']
-------------------------------
[2017]
['BUL', 'BUL', 'JPN', 'CAN', 'BUL']
-------------------------------
[2003]
['BEL', 'GER', 'CZE', 'GER', 'CZE']
-------------------------------
[2007]
['ROU', 'FIN', 'GER', 'FRA', 'CZE']
-------------------------------
[2009]
['FRA', 'AUT', 'FRA', 'AUS', 'FRA']
-------------------------------
[2015]
['SUI', 'JPN', 'USA', 'POR', 'FRA']
-------------------------------
[2012]
['USA', 'LTU', 'BE

# Winner Count per country per year

In [ ]:
winning_country_year_df=[] #We will have seperate dataframe for every year, all dataframe to be stored in this list
year_key = {}
count = 0

for year in set(list(tennis_df['year'])): #Looping through countries
    tmp_list = []
    for country in set(list(tennis_df[tennis_df['year'] == year]['winner_ioc'])): #Looping through years
      if country != None:
        tmp_list.append([country,len(tennis_df[(tennis_df['winner_ioc'] == country) & (tennis_df['year'] == year)])]) #Addding country name and number of winners
    winning_country_year_df.append(pd.DataFrame(tmp_list,columns=['Country','Winner_Count'])) #converting to a dataframe and storing to list
    year_key[count] = year  #Creating index to year dictionary for future reference
    count+=1

In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import geopandas as gpd #For chrolopleth

In [ ]:
import geopandas as gpd
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))  #This file has geography information
print(world.head())
world['new_name']=world['name'].str.upper().apply(lambda y: y[0:3])  #creating a new column with first 3 letters of country name

     pop_est  ...                                           geometry
0     920938  ...  MULTIPOLYGON (((180.00000 -16.06713, 180.00000...
1   53950935  ...  POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...
2     603253  ...  POLYGON ((-8.66559 27.65643, -8.66512 27.58948...
3   35623680  ...  MULTIPOLYGON (((-122.84000 49.00000, -122.9742...
4  326625791  ...  MULTIPOLYGON (((-122.84000 49.00000, -120.0000...

[5 rows x 6 columns]


In [ ]:
world.to_csv("/content/ATP_matches_Visualisation/dataset/world.csv") #Downloading for analysis via excel

In [ ]:
#
winning_country_year_geometry_df = [] #List to hold dataframe for every year. Each dataframe will be an extension of world df with winner count as additional column

#Below are some hard coded values to map with the ones in world dataframe
#
unavailable_Country_codes = [] #To save country codes which are not available

#AHO belong to netherlands antilles, which is not available in world dataframe hence mapping it to USA
#GUM belong to Guam, which is not available in world dataframe hence mapping it to USA
#AND belong to Andorra, which is not available in world dataframe hence mapping it to spain
#BAR belong to Barbados, which is not available in world dataframe hence mapping it to PRI
#HKG belong to Hong Kong, which is not available in world dataframe hence mapping it to CHINA - CHN
#MHL belong to Marshall Islands, which is not available in world dataframe hence mapping it to USA
#SAM belong to Samoa, which is not available in world dataframe hence mapping it to USA
#COK belong to cook islands, which is not available in world dataframe hence mapping it to NZL

#Dictionary to map ATP country code to ISO country code for the ones which are tricky
some_known_dict = {'INA':'IDN','PUR':'PRI','ESA':'EST','RSA':'ZAF','MAS':'MYS','NED':'NLD','TPE':'TWN','SUI':'CHE','AHO':'USA'\
                   ,'AND':'ESP','CRC':'CRI','GUM':'USA','HKG':'CHN','IRI':'IRN','MHL':'USA','SAM':'USA','BAR':'PRI','UAE':'ARE',\
                   'NGR':'NGA','COK':'NZL'}

for e in range(len(winning_country_year_df)):
  geom_col = []  #We want to create a new column which will have the iso_a3 country code for a given country
  for each_count in list(winning_country_year_df[e]['Country']): #each_count is each_country
    subsetdf = world[world['iso_a3'] == each_count] #Checking is the ATP country code is same as iso country code
    if len(subsetdf) == 1:
      geom_col.append(subsetdf.iloc[0]['iso_a3'])  #Appending the iso country code
    else:
      subsetdf = world[world['new_name'] == each_count]  #We created a new columns with the first 3 letters of every country. Checking to see if it matches ATP code
      if len(subsetdf) > 0:
        geom_col.append(subsetdf.iloc[0]['iso_a3']) #In case of a match, Appending its iso country code
      else:
        try:
          subsetdf = world[world['iso_a3'] == some_known_dict[each_count]] #Else checking in out dictonary for the ATP country code
          if len(subsetdf) > 0:
            geom_col.append(some_known_dict[each_count])  #In case of a match appending its iso_a3 country code
          else:
            print(each_count," could not be found")  #Else printing to check manually
            unavailable_Country_codes.append(each_count)
            print(e)
            geom_col.append("")
        except:  #Else printing to check manually
          print(each_count," could not be found")
          unavailable_Country_codes.append(each_count)
          print(e)
          geom_col.append("")
  winning_country_year_df[e]['matching_col'] = geom_col  #Appending the new column to the dataframe
  tmp_world_df = world.merge(winning_country_year_df[e], how='left', left_on="iso_a3", right_on="matching_col")
  tmp_world_df['Winner_Count'] = tmp_world_df['Winner_Count'].fillna(0)
  winning_country_year_geometry_df.append(tmp_world_df) #Appending the winner count to the world dataframe using the newly created column and then appending them to a list
unavailable_Country_codes =set(unavailable_Country_codes)
if len(unavailable_Country_codes) > 0:
  print("Unavailable country codes",unavailable_Country_codes)

# Choloropleth Animation

In [ ]:
output_path = '/content/ATP_matches_Visualisation/maps'
try:
  os.mkdir(output_path)
except:
  print("unable to create output directory "+output_path)

images = []

vmin, vmax = 0, 370
for e in range(len(winning_country_year_geometry_df)):
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = winning_country_year_geometry_df[e].plot(column='Winner_Count', cmap='Blues', figsize=(10,10), linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax,
legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax), legend_kwds={'shrink': 0.5})
    
   
    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title('Winners of ATP matches', \
              fontdict={'fontsize': '18',
                         'fontweight' : '3'})
    fig.annotate(year_key[e],
            xy=(0.1, .35), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=18)
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, str(year_key[e])+'_ATP_winners.png')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=300, transparent=True)



from PIL import Image
for e in range(len(winning_country_year_geometry_df)):
  im = Image.open(os.path.join(output_path, str(year_key[e])+'_ATP_winners.png'))
  images.append(im)

# creating the GIF
images[0].save(output_path+'/ATP_matches_winners.gif',
               save_all=True, append_images=images[1:], optimize=True, duration=800, loop=0)

#Standarding...........


In [ ]:
def show_gif(fname):
    import base64
    from IPython import display
    with open(fname, 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
    return display.HTML(f'<img src="data:image/gif;base64,{b64}" width="750" align="center" />')
show_gif('/content/ATP_matches_Visualisation/maps/ATP_matches_winners.gif')

# Loser Count Per Country

In [ ]:
winning_country_year_df=[] #We will have seperate dataframe for every year, all dataframe to be stored in this list
year_key = {}
count = 0

for year in set(list(tennis_df['year'])): #Looping through countries
    tmp_list = []
    for country in set(list(tennis_df[tennis_df['year'] == year]['loser_ioc'])): #Looping through years
      if country != None:
        tmp_list.append([country,len(tennis_df[(tennis_df['loser_ioc'] == country) & (tennis_df['year'] == year)])]) #Addding country name and number of winners
    winning_country_year_df.append(pd.DataFrame(tmp_list,columns=['Country','Winner_Count'])) #converting to a dataframe and storing to list
    year_key[count] = year  #Creating index to year dictionary for future reference
    count+=1

#
winning_country_year_geometry_df = [] #List to hold dataframe for every year. Each dataframe will be an extension of world df with winner count as additional column

#Below are some hard coded values to map with the ones in world dataframe
#
unavailable_Country_codes = [] #To save country codes which are not available

#AHO belong to netherlands antilles, which is not available in world dataframe hence mapping it to USA
#GUM belong to Guam, which is not available in world dataframe hence mapping it to USA
#AND belong to Andorra, which is not available in world dataframe hence mapping it to spain
#BAR belong to Barbados, which is not available in world dataframe hence mapping it to PRI
#HKG belong to Hong Kong, which is not available in world dataframe hence mapping it to CHINA - CHN
#MHL belong to Marshall Islands, which is not available in world dataframe hence mapping it to USA
#SAM belong to Samoa, which is not available in world dataframe hence mapping it to USA
#COK belong to cook islands, which is not available in world dataframe hence mapping it to NZL

#Dictionary to map ATP country code to ISO country code for the ones which are tricky
some_known_dict = {'INA':'IDN','PUR':'PRI','ESA':'EST','RSA':'ZAF','MAS':'MYS','NED':'NLD','TPE':'TWN','SUI':'CHE','AHO':'USA'\
                   ,'AND':'ESP','CRC':'CRI','GUM':'USA','HKG':'CHN','IRI':'IRN','MHL':'USA','SAM':'USA','BAR':'PRI','UAE':'ARE',\
                   'NGR':'NGA','COK':'NZL'}

for e in range(len(winning_country_year_df)):
  geom_col = []  #We want to create a new column which will have the iso_a3 country code for a given country
  for each_count in list(winning_country_year_df[e]['Country']): #each_count is each_country
    subsetdf = world[world['iso_a3'] == each_count] #Checking is the ATP country code is same as iso country code
    if len(subsetdf) == 1:
      geom_col.append(subsetdf.iloc[0]['iso_a3'])  #Appending the iso country code
    else:
      subsetdf = world[world['new_name'] == each_count]  #We created a new columns with the first 3 letters of every country. Checking to see if it matches ATP code
      if len(subsetdf) > 0:
        geom_col.append(subsetdf.iloc[0]['iso_a3']) #In case of a match, Appending its iso country code
      else:
        try:
          subsetdf = world[world['iso_a3'] == some_known_dict[each_count]] #Else checking in out dictonary for the ATP country code
          if len(subsetdf) > 0:
            geom_col.append(some_known_dict[each_count])  #In case of a match appending its iso_a3 country code
          else:
            print(each_count," could not be found")  #Else printing to check manually
            unavailable_Country_codes.append(each_count)
            print(e)
            geom_col.append("")
        except:  #Else printing to check manually
          print(each_count," could not be found")
          unavailable_Country_codes.append(each_count)
          print(e)
          geom_col.append("")
  winning_country_year_df[e]['matching_col'] = geom_col  #Appending the new column to the dataframe
  tmp_world_df = world.merge(winning_country_year_df[e], how='left', left_on="iso_a3", right_on="matching_col")
  tmp_world_df['Winner_Count'] = tmp_world_df['Winner_Count'].fillna(0)
  winning_country_year_geometry_df.append(tmp_world_df) #Appending the winner count to the world dataframe using the newly created column and then appending them to a list
unavailable_Country_codes =set(unavailable_Country_codes)
if len(unavailable_Country_codes) > 0:
  print("Unavailable country codes",unavailable_Country_codes)

output_path = '/content/ATP_matches_Visualisation/maps'
try:
  os.mkdir(output_path)
except:
  print("unable to create output directory "+output_path)

images = []

vmin, vmax = 0, 370
for e in range(len(winning_country_year_geometry_df)):
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = winning_country_year_geometry_df[e].plot(column='Winner_Count', cmap='Blues', figsize=(10,10), linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax,
legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax), legend_kwds={'shrink': 0.5})
    
   
    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title('Losers of ATP matches', \
              fontdict={'fontsize': '18',
                         'fontweight' : '3'})
    fig.annotate(year_key[e],
            xy=(0.1, .35), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=18)
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, str(year_key[e])+'_ATP_losers.png')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=300, transparent=True)


from PIL import Image
for e in range(len(winning_country_year_geometry_df)):
  im = Image.open(os.path.join(output_path, str(year_key[e])+'_ATP_losers.png'))
  images.append(im)

# creating the GIF
images[0].save(output_path+'/ATP_matches_losers.gif',
               save_all=True, append_images=images[1:], optimize=True, duration=800, loop=0)



In [ ]:
#Standarding...........
show_gif('/content/ATP_matches_Visualisation/maps/ATP_matches_losers.gif')

# Total players in a country

In [ ]:

from collections import defaultdict

country_dict = defaultdict(set)
for k, g in tennis_df.groupby(["winner_name", "winner_ioc"]):
    country_dict[k[1]].add(k[0])
for k, g in tennis_df.groupby(["loser_name", "loser_ioc"]):
    country_dict[k[1]].add(k[0])
out = [[k, len(country_dict[k])]for k in country_dict]
player_count = pd.DataFrame(out, columns=["country","no_players"])

# Winning Count - Normalised

In [ ]:


winning_country_year_df=[] #We will have seperate dataframe for every year, all dataframe to be stored in this list
year_key = {}
count = 0

for year in set(list(tennis_df['year'])): #Looping through countries
    tmp_list = []
    country_dict = defaultdict(set)
    for k, g in tennis_df[tennis_df['year'] == year].groupby(["winner_name", "winner_ioc"]):
        country_dict[k[1]].add(k[0])
    out = [[k, len(country_dict[k])]for k in country_dict]
    winning_country_year_df.append(pd.DataFrame(out, columns=["Country","Winner_Count"]))
    year_key[count] = year  #Creating index to year dictionary for future reference
    count+=1

for e in range(len(winning_country_year_df)):
  for country in list(winning_country_year_df[e]['Country']):
    winning_country_year_df[e].loc[winning_country_year_df[e]['Country'] == country,'Winner_Count']/=np.array(player_count[player_count['country']==country]['no_players'])[0]
    #break
  #break


In [ ]:
'''winning_country_year_df=[] #We will have seperate dataframe for every year, all dataframe to be stored in this list
year_key = {}
count = 0

for year in set(list(tennis_df['year'])): #Looping through countries
    tmp_list = []
    for country in set(list(tennis_df[tennis_df['year'] == year]['winner_ioc'])): #Looping through years
      if country != None:
        #print(np.array(player_count[player_count['country']==country]['no_players'])[0])
        #print(len(tennis_df[(tennis_df['winner_ioc'] == country) & (tennis_df['year'] == year)]))
        #print("-------------------------------")
        tmp_list.append([country,len(tennis_df[(tennis_df['winner_ioc'] == country) & (tennis_df['year'] == year)]) /np.array(player_count[player_count['country']==country]['no_players'])[0] ]) #Addding country name and number of winners
    winning_country_year_df.append(pd.DataFrame(tmp_list,columns=['Country','Winner_Count'])) #converting to a dataframe and storing to list
    year_key[count] = year  #Creating index to year dictionary for future reference
    count+=1'''

'winning_country_year_df=[] #We will have seperate dataframe for every year, all dataframe to be stored in this list\nyear_key = {}\ncount = 0\n\nfor year in set(list(tennis_df[\'year\'])): #Looping through countries\n    tmp_list = []\n    for country in set(list(tennis_df[tennis_df[\'year\'] == year][\'winner_ioc\'])): #Looping through years\n      if country != None:\n        #print(np.array(player_count[player_count[\'country\']==country][\'no_players\'])[0])\n        #print(len(tennis_df[(tennis_df[\'winner_ioc\'] == country) & (tennis_df[\'year\'] == year)]))\n        #print("-------------------------------")\n        tmp_list.append([country,len(tennis_df[(tennis_df[\'winner_ioc\'] == country) & (tennis_df[\'year\'] == year)]) /np.array(player_count[player_count[\'country\']==country][\'no_players\'])[0] ]) #Addding country name and number of winners\n    winning_country_year_df.append(pd.DataFrame(tmp_list,columns=[\'Country\',\'Winner_Count\'])) #converting to a dataframe and 

In [ ]:
#
winning_country_year_geometry_df = [] #List to hold dataframe for every year. Each dataframe will be an extension of world df with winner count as additional column

#Below are some hard coded values to map with the ones in world dataframe
#
unavailable_Country_codes = [] #To save country codes which are not available

#AHO belong to netherlands antilles, which is not available in world dataframe hence mapping it to USA
#GUM belong to Guam, which is not available in world dataframe hence mapping it to USA
#AND belong to Andorra, which is not available in world dataframe hence mapping it to spain
#BAR belong to Barbados, which is not available in world dataframe hence mapping it to PRI
#HKG belong to Hong Kong, which is not available in world dataframe hence mapping it to CHINA - CHN
#MHL belong to Marshall Islands, which is not available in world dataframe hence mapping it to USA
#SAM belong to Samoa, which is not available in world dataframe hence mapping it to USA
#COK belong to cook islands, which is not available in world dataframe hence mapping it to NZL

#Dictionary to map ATP country code to ISO country code for the ones which are tricky
some_known_dict = {'INA':'IDN','PUR':'PRI','ESA':'EST','RSA':'ZAF','MAS':'MYS','NED':'NLD','TPE':'TWN','SUI':'CHE','AHO':'USA'\
                   ,'AND':'ESP','CRC':'CRI','GUM':'USA','HKG':'CHN','IRI':'IRN','MHL':'USA','SAM':'USA','BAR':'PRI','UAE':'ARE',\
                   'NGR':'NGA','COK':'NZL'}

for e in range(len(winning_country_year_df)):
  geom_col = []  #We want to create a new column which will have the iso_a3 country code for a given country
  for each_count in list(winning_country_year_df[e]['Country']): #each_count is each_country
    subsetdf = world[world['iso_a3'] == each_count] #Checking is the ATP country code is same as iso country code
    if len(subsetdf) == 1:
      geom_col.append(subsetdf.iloc[0]['iso_a3'])  #Appending the iso country code
    else:
      subsetdf = world[world['new_name'] == each_count]  #We created a new columns with the first 3 letters of every country. Checking to see if it matches ATP code
      if len(subsetdf) > 0:
        geom_col.append(subsetdf.iloc[0]['iso_a3']) #In case of a match, Appending its iso country code
      else:
        try:
          subsetdf = world[world['iso_a3'] == some_known_dict[each_count]] #Else checking in out dictonary for the ATP country code
          if len(subsetdf) > 0:
            geom_col.append(some_known_dict[each_count])  #In case of a match appending its iso_a3 country code
          else:
            print(each_count," could not be found")  #Else printing to check manually
            unavailable_Country_codes.append(each_count)
            print(e)
            geom_col.append("")
        except:  #Else printing to check manually
          print(each_count," could not be found")
          unavailable_Country_codes.append(each_count)
          print(e)
          geom_col.append("")
  winning_country_year_df[e]['matching_col'] = geom_col  #Appending the new column to the dataframe
  tmp_world_df = world.merge(winning_country_year_df[e], how='left', left_on="iso_a3", right_on="matching_col")
  tmp_world_df['Winner_Count'] = tmp_world_df['Winner_Count'].fillna(0)
  winning_country_year_geometry_df.append(tmp_world_df) #Appending the winner count to the world dataframe using the newly created column and then appending them to a list
unavailable_Country_codes =set(unavailable_Country_codes)
if len(unavailable_Country_codes) > 0:
  print("Unavailable country codes",unavailable_Country_codes)
  
output_path = '/content/ATP_matches_Visualisation/maps'
try:
  os.mkdir(output_path)
except:
  print("unable to create output directory "+output_path)

images = []

vmin, vmax = 0, 1
for e in range(len(winning_country_year_geometry_df)):
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = winning_country_year_geometry_df[e].plot(column='Winner_Count', cmap='Blues', figsize=(10,10), linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax,
legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax), legend_kwds={'shrink': 0.5})
    
   
    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title('Winners (to total players) of ATP matches', \
              fontdict={'fontsize': '18',
                         'fontweight' : '3'})
    fig.annotate(year_key[e],
            xy=(0.1, .35), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=18)
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, str(year_key[e])+'_ATP_winners_ratio.png')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=300, transparent=True)


from PIL import Image
for e in range(len(winning_country_year_geometry_df)):
  im = Image.open(os.path.join(output_path, str(year_key[e])+'_ATP_winners_ratio.png'))
  images.append(im)

# creating the GIF
images[0].save(output_path+'/ATP_matches_winners_noramlized.gif',
               save_all=True, append_images=images[1:], optimize=True, duration=800, loop=0)




In [ ]:
show_gif('/content/ATP_matches_Visualisation/maps/ATP_matches_winners_noramlized.gif')

# Losers Count - Normalised

In [ ]:



winning_country_year_df=[] #We will have seperate dataframe for every year, all dataframe to be stored in this list
year_key = {}
count = 0

for year in set(list(tennis_df['year'])): #Looping through countries
    tmp_list = []
    country_dict = defaultdict(set)
    for k, g in tennis_df[tennis_df['year'] == year].groupby(["loser_name", "loser_ioc"]):
        country_dict[k[1]].add(k[0])
    out = [[k, len(country_dict[k])]for k in country_dict]
    winning_country_year_df.append(pd.DataFrame(out, columns=["Country","Loser_Count"]))
    year_key[count] = year  #Creating index to year dictionary for future reference
    count+=1

for e in range(len(winning_country_year_df)):
  for country in list(winning_country_year_df[e]['Country']):
    winning_country_year_df[e].loc[winning_country_year_df[e]['Country'] == country,'Loser_Count']/=np.array(player_count[player_count['country']==country]['no_players'])[0]
    #break
  #break




winning_country_year_geometry_df = [] #List to hold dataframe for every year. Each dataframe will be an extension of world df with winner count as additional column

#Below are some hard coded values to map with the ones in world dataframe
#
unavailable_Country_codes = [] #To save country codes which are not available

#AHO belong to netherlands antilles, which is not available in world dataframe hence mapping it to USA
#GUM belong to Guam, which is not available in world dataframe hence mapping it to USA
#AND belong to Andorra, which is not available in world dataframe hence mapping it to spain
#BAR belong to Barbados, which is not available in world dataframe hence mapping it to PRI
#HKG belong to Hong Kong, which is not available in world dataframe hence mapping it to CHINA - CHN
#MHL belong to Marshall Islands, which is not available in world dataframe hence mapping it to USA
#SAM belong to Samoa, which is not available in world dataframe hence mapping it to USA
#COK belong to cook islands, which is not available in world dataframe hence mapping it to NZL

#Dictionary to map ATP country code to ISO country code for the ones which are tricky
some_known_dict = {'INA':'IDN','PUR':'PRI','ESA':'EST','RSA':'ZAF','MAS':'MYS','NED':'NLD','TPE':'TWN','SUI':'CHE','AHO':'USA'\
                   ,'AND':'ESP','CRC':'CRI','GUM':'USA','HKG':'CHN','IRI':'IRN','MHL':'USA','SAM':'USA','BAR':'PRI','UAE':'ARE',\
                   'NGR':'NGA','COK':'NZL'}

for e in range(len(winning_country_year_df)):
  geom_col = []  #We want to create a new column which will have the iso_a3 country code for a given country
  for each_count in list(winning_country_year_df[e]['Country']): #each_count is each_country
    subsetdf = world[world['iso_a3'] == each_count] #Checking is the ATP country code is same as iso country code
    if len(subsetdf) == 1:
      geom_col.append(subsetdf.iloc[0]['iso_a3'])  #Appending the iso country code
    else:
      subsetdf = world[world['new_name'] == each_count]  #We created a new columns with the first 3 letters of every country. Checking to see if it matches ATP code
      if len(subsetdf) > 0:
        geom_col.append(subsetdf.iloc[0]['iso_a3']) #In case of a match, Appending its iso country code
      else:
        try:
          subsetdf = world[world['iso_a3'] == some_known_dict[each_count]] #Else checking in out dictonary for the ATP country code
          if len(subsetdf) > 0:
            geom_col.append(some_known_dict[each_count])  #In case of a match appending its iso_a3 country code
          else:
            print(each_count," could not be found")  #Else printing to check manually
            unavailable_Country_codes.append(each_count)
            print(e)
            geom_col.append("")
        except:  #Else printing to check manually
          print(each_count," could not be found")
          unavailable_Country_codes.append(each_count)
          print(e)
          geom_col.append("")
  winning_country_year_df[e]['matching_col'] = geom_col  #Appending the new column to the dataframe
  tmp_world_df = world.merge(winning_country_year_df[e], how='left', left_on="iso_a3", right_on="matching_col")
  tmp_world_df['Loser_Count'] = tmp_world_df['Loser_Count'].fillna(0)
  winning_country_year_geometry_df.append(tmp_world_df) #Appending the winner count to the world dataframe using the newly created column and then appending them to a list
unavailable_Country_codes =set(unavailable_Country_codes)
if len(unavailable_Country_codes) > 0:
  print("Unavailable country codes",unavailable_Country_codes)
  
output_path = '/content/ATP_matches_Visualisation/maps'
try:
  os.mkdir(output_path)
except:
  print("unable to create output directory "+output_path)

images = []

vmin, vmax = 0, 1
for e in range(len(winning_country_year_geometry_df)):
    
    # create map, UDPATE: added plt.Normalize to keep the legend range the same for all maps
    fig = winning_country_year_geometry_df[e].plot(column='Loser_Count', cmap='Blues', figsize=(10,10), linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax,
legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax), legend_kwds={'shrink': 0.5})
    
   
    # remove axis of chart
    fig.axis('off')
    
    # add a title
    fig.set_title('Losers (to total players)of ATP matches', \
              fontdict={'fontsize': '18',
                         'fontweight' : '3'})
    fig.annotate(year_key[e],
            xy=(0.1, .35), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=18)
    # this will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    filepath = os.path.join(output_path, str(year_key[e])+'_ATP_losers_ratio.png')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=300, transparent=True)


from PIL import Image
for e in range(len(winning_country_year_geometry_df)):
  im = Image.open(os.path.join(output_path, str(year_key[e])+'_ATP_losers_ratio.png'))
  images.append(im)

# creating the GIF
images[0].save(output_path+'/ATP_matches_losers_noramlized.gif',
               save_all=True, append_images=images[1:], optimize=True, duration=800, loop=0)




In [ ]:
show_gif('/content/ATP_matches_Visualisation/maps/ATP_matches_losers_noramlized.gif')

# References

https://towardsdatascience.com/how-to-make-a-gif-map-using-python-geopandas-and-matplotlib-cd8827cefbc8

https://medium.com/@maeliza.seymour/animated-choropleth-map-with-plotly-covid-19-use-case-7a13244d0ee3

https://www.wikiwand.com/en/List_of_IOC_country_codes - ATP country codes